This program reads in the aggregated data file and modifies it with using dictionaries

0. Reading in the pervious made, combined file

In [1]:
#readnig in the combined file
with open('inputs/combinedtext/combinedalldata.txt', 'r',encoding='utf-8') as fp: 
    combinedalldata = fp.read() 
#print(dataossz)

1. solve all abbreviations #there will be only puncts there, where it is an end of sentence

In [2]:
#load the abbreviations dictionarie and build a python dictionary
abbreviationsdict = dict() #make an empty dict
with open('inputs/dictionaries/abbreviations.txt', 'r',encoding='utf-8') as fp: 
    next(fp) #skip the first line because there are the instructions
    for line in fp:
        wordsarray = line.split('=')
        abbreviationsdict[wordsarray[0]] = wordsarray[1][:-1] #\n minus

        
combinedalldatawithoutabbreviations=combinedalldata

i=0            
while i<len(abbreviationsdict):
    combinedalldatawithoutabbreviations=combinedalldatawithoutabbreviations.replace(list(abbreviationsdict.keys())[i],list(abbreviationsdict.values())[i])
    i=i+1
    
#elements of dict    
# list(roviditesszotar.keys())[1]
# list(roviditesszotar.items())[1] 


In [4]:
abbreviationsdict

{'ill.': 'illetve',
 'abs.': 'absolut',
 'ápr.': 'április',
 'ált.': 'általában',
 'aug.': 'augusztus',
 'Bp.': 'Budapest',
 'dec.': 'december',
 'eü.': 'egészségügy',
 'febr.': 'február',
 'gjmű.': 'gépjármű',
 'jan.': 'január',
 'jún.': 'június',
 'jmű.': 'jármű',
 'máj.': 'május',
 'márc.': 'március',
 'min.': 'minimum',
 'Mo.': 'Magyarország',
 'nov.': 'november',
 'okt.': 'október'}

In [3]:
with open('outputs/preprocess/combdatwithoutabbr.txt','w',encoding='utf-8') as output:
    output.write(combinedalldatawithoutabbreviations)


3. delete all spaces before \n-s or delete all douple \n-s 

In [4]:
combinedalldatawithoutparagraphs=combinedalldatawithoutabbreviations
i=0
while(i<100):
    combinedalldatawithoutparagraphs=combinedalldatawithoutparagraphs.replace(' \n','\n')
    combinedalldatawithoutparagraphs=combinedalldatawithoutparagraphs.replace('\n\n','\n')
    i=i+1

In [5]:
with open('outputs/preprocess/combdatwithoutparag.txt','w',encoding='utf-8') as output:
    output.write(combinedalldatawithoutparagraphs)

4. search and repleace .-s to <pont> 
    goals: each snetence to different raw
    technik: 1 find .  check if after2character of . if it is capital 3 if yes check if the caracter after. is \n if not->change 
    (i dont change every . to <pont> here because later I will be able to find the "wrong ones which are not end of sentence neither abbrevation

In [6]:
#the function tho change caracters in string
def replace_str_index(text,index=0,replacement=''):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])

In [7]:
#loading bar function
from __future__ import print_function
import sys
import re


class ProgressBar(object):
    DEFAULT = 'Progress: %(bar)s %(percent)3d%%'
    FULL = '%(bar)s %(current)d/%(total)d (%(percent)3d%%) %(remaining)d to go'

    def __init__(self, total, width=40, fmt=DEFAULT, symbol='=',
                 output=sys.stderr):
        assert len(symbol) == 1

        self.total = total
        self.width = width
        self.symbol = symbol
        self.output = output
        self.fmt = re.sub(r'(?P<name>%\(.+?\))d',
            r'\g<name>%dd' % len(str(total)), fmt)

        self.current = 0

    def __call__(self):
        percent = self.current / float(self.total)
        size = int(self.width * percent)
        remaining = self.total - self.current
        bar = '[' + self.symbol * size + ' ' * (self.width - size) + ']'

        args = {
            'total': self.total,
            'bar': bar,
            'current': self.current,
            'percent': percent * 100,
            'remaining': remaining
        }
        print('\r' + self.fmt % args, file=self.output, end='')

    def done(self):
        self.current = self.total
        self()
        print('', file=self.output)

In [11]:
cadsentenceinnewline=combinedalldatawithoutparagraphs

In [26]:
#find places of points

progress=ProgressBar(len(cadsentenceinnewline),fmt=ProgressBar.FULL)
index=0
iold=0
progress.current=0
while(cadsentenceinnewline.find('.', index+1)>0):
#     progress.current += 1
#     progress()
    index=cadsentenceinnewline.find('.',index+1)
 #   print(index)
 #   print(bekezdesfeloldottossz[index])
    
    if(index>(10000+iold)) :
        iold=index
        progress.current = index
        progress()
    #check if the 2. after . is capital
    while(cadsentenceinnewline[index+2].isupper() or cadsentenceinnewline[index+2]=='('):
        #if yes ->it is end of sentence 
        #is there \n after . ?
        if(cadsentenceinnewline[index+1]=='\n'):
            cadsentenceinnewline=replace_str_index(cadsentenceinnewline,index,' <pont>')
            break
        #if the caracter is not \n but space ill change it and breakit into newline 
        elif(cadsentenceinnewline[index+1]==' '):
            cadsentenceinnewline=replace_str_index(cadsentenceinnewline,index+1,'\n')
            cadsentenceinnewline=replace_str_index(cadsentenceinnewline,index,' <pont>')
            break
        else:
#             print("oops, I dont kow what does this point mean" + ' ' + str(index))
            break


[===                                     ]  1276532/16936814 (  7%) 15660282 to go

[=========================================] 17641656/16936814 (104%)  -704842 to go

IndexError: string index out of range

In [ ]:
#cadsentenceinnewline

In [29]:
progress.current=0
with open('outputs/preprocess/cadsentenceinnewline.txt','w',encoding='utf-8') as output:
    output.write(cadsentenceinnewline)

5. change words with dict 2

In [30]:
#load and create dict 

punctdict = dict() #emptydict
with open('inputs/dictionaries//punctuations.txt', 'r',encoding='utf-8') as fp: 
    next(fp) #skip the first line because there are the instructions
    for line in fp:
        wordsarray = line.split('=')
        punctdict[wordsarray[0]] = wordsarray[1][:-1] #minus \n
        
cadwordschanged=cadsentenceinnewline

i=0            
while i<len(punctdict):
    cadwordschanged=cadwordschanged.replace(list(punctdict.keys())[i],list(punctdict.values())[i])
    i=i+1
    

In [31]:
punctdict

{'-': ' <kötõjel> ',
 ',': ' <vesszõ> ',
 ':': ' <kettõspont> ',
 '(': ' <zárójelnyit> ',
 ')': ' <zárójelzár> ',
 '.': ' <pont> ',
 '*': ' ',
 '?': ' <kérdőjel>'}

6. make all letters low

In [32]:
cadlowletter=cadwordschanged.lower()

In [34]:
with open('outputs/preprocess/cadlowletter.txt','w',encoding='utf-8') as output:
    output.write(cadlowletter)

4 save

In [35]:
with open('outputs/endoutput/prepareddata.txt','w',encoding='utf-8') as output:
    output.write(cadlowletter)

7 delete repetative words if repeating 3 times next to eachother
    1. method: read in every raw.
    2. split to words
    3. delete repetative words
    4. join the words and write out.

In [36]:
# with open('outputs/endoutput/prepareddata.txt', 'r',encoding='utf-8') as fp: 
#     line = fp.readline()
#     print(line)
#     line = fp.readline()
#     print(line)
#     line = fp.readline()
#     print(line)

In [1]:
with open('outputs/endoutput/prepareddata.txt', 'r',encoding='utf-8') as fp: 
    lines = fp.readlines()

In [2]:
lines2=[]
for line in lines: 
    line=line.split()
    i=2
    while(i < len(line)) : 
        if(line[i]==line[i-1]==line[i-2]): #if repetative 3 
            del line[i]
            i=i-1
        i=i+1
    line=" ".join(line)
    lines2.append(line+"\n")
lines2=''.join(lines2)

In [3]:
with open("outputs/endoutput/prepareddata2.txt", "w") as outfile:
    outfile.write(lines2)
    
#     with open("outfile", "w") as outfile:
#     outfile.write("\n".join(itemlist))

build 
train
valid
test
database
(70-20-10%)
It is enough to cut only, because the texts were randomised by the merging phase

In [4]:
tokenisator=lines2
databasetrain=lines2[0:int(len(lines2)*0.7)] #cut the word list at 0-70%
databasevalid=lines2[int(len(lines2)*0.7):int(len(lines2)*0.9)] #cut the word list at 70%-90%
databasetest=lines2[int(len(lines2)*0.9):] #cut the word list at 90%-

In [5]:
with open("outputs/endoutput/databasetrain.txt", "w") as outfile:
    outfile.write(databasetrain)
    
with open("outputs/endoutput/databasevalid.txt", "w") as outfile:
    outfile.write(databasevalid)
    
with open("outputs/endoutput/databasetest.txt", "w") as outfile:
    outfile.write(databasetest)